<a href="https://colab.research.google.com/github/suheon927/PerceptronPractice/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# XOR 데이터셋
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# 활성화 함수
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# 다층 퍼셉트론 모델
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # 가중치 초기화
        self.weights_input_hidden = np.random.uniform(-0.5, 0.5, (input_size, hidden_size))
        self.weights_hidden_output = np.random.uniform(-0.5, 0.5, (hidden_size, output_size))

        # 바이어스 초기화
        self.bias_hidden = np.zeros((1, hidden_size))
        self.bias_output = np.zeros((1, output_size))

        # Adam 옵티마이저 변수
        self.m_weights_ih = np.zeros_like(self.weights_input_hidden)
        self.v_weights_ih = np.zeros_like(self.weights_input_hidden)
        self.m_weights_ho = np.zeros_like(self.weights_hidden_output)
        self.v_weights_ho = np.zeros_like(self.weights_hidden_output)
        self.m_bias_h = np.zeros_like(self.bias_hidden)
        self.v_bias_h = np.zeros_like(self.bias_hidden)
        self.m_bias_o = np.zeros_like(self.bias_output)
        self.v_bias_o = np.zeros_like(self.bias_output)

        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8
        self.learning_rate = 0.01
        self.t = 0

    # 순전파
    def forward(self, x):
        self.hidden_input = np.dot(x, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = sigmoid(self.final_input)
        return self.final_output

    # 역전파 (Adam 옵티마이저 포함)
    def backward(self, x, y, output):
        self.t += 1
        output_error = y - output  # 출력층 오차
        output_delta = output_error * sigmoid_derivative(output)

        hidden_error = np.dot(output_delta, self.weights_hidden_output.T)  # 은닉층 오차
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden_output)

        # 가중치와 바이어스의 변화량 계산
        weights_ho_grad = np.dot(self.hidden_output.T, output_delta)
        weights_ih_grad = np.dot(x.T, hidden_delta)
        bias_o_grad = np.sum(output_delta, axis=0, keepdims=True)
        bias_h_grad = np.sum(hidden_delta, axis=0, keepdims=True)

        # Adam 옵티마이저를 이용한 가중치 업데이트
        self.update_with_adam(self.weights_input_hidden, self.m_weights_ih, self.v_weights_ih, weights_ih_grad)
        self.update_with_adam(self.weights_hidden_output, self.m_weights_ho, self.v_weights_ho, weights_ho_grad)
        self.update_with_adam(self.bias_hidden, self.m_bias_h, self.v_bias_h, bias_h_grad)
        self.update_with_adam(self.bias_output, self.m_bias_o, self.v_bias_o, bias_o_grad)

    # Adam 옵티마이저 업데이트
    def update_with_adam(self, weights, m, v, grad):
        m[:] = self.beta1 * m + (1 - self.beta1) * grad
        v[:] = self.beta2 * v + (1 - self.beta2) * (grad ** 2)
        m_hat = m / (1 - self.beta1 ** self.t)
        v_hat = v / (1 - self.beta2 ** self.t)
        weights += self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)

    # 훈련 함수
    def train(self, x, y, epochs):
        for epoch in range(epochs):
            output = self.forward(x)
            self.backward(x, y, output)
            if epoch % 1000 == 0:
                loss = np.mean((y - output) ** 2)
                print(f"Epoch {epoch}, Loss: {loss}")

# 모델 초기화
mlp = MLP(input_size=2, hidden_size=16, output_size=1)

# 모델 훈련
mlp.train(x, y, epochs=10000)

# 예측 결과 확인
for i in range(len(x)):
    print(f"Input: {x[i]}, Predicted Output: {mlp.forward(x[i])}, Actual Output: {y[i]}")

Epoch 0, Loss: 0.2889309748103554
Epoch 1000, Loss: 0.0004065805583503611
Epoch 2000, Loss: 8.53528386672547e-05
Epoch 3000, Loss: 3.338421515273317e-05
Epoch 4000, Loss: 1.603877026172857e-05
Epoch 5000, Loss: 8.479525972913184e-06
Epoch 6000, Loss: 4.718800666068435e-06
Epoch 7000, Loss: 2.7049298170991077e-06
Epoch 8000, Loss: 1.5786825888267907e-06
Epoch 9000, Loss: 9.31893504010189e-07
Input: [0 0], Predicted Output: [[0.00082715]], Actual Output: [0]
Input: [0 1], Predicted Output: [[0.99926735]], Actual Output: [1]
Input: [1 0], Predicted Output: [[0.99923039]], Actual Output: [1]
Input: [1 1], Predicted Output: [[0.00063522]], Actual Output: [0]
